In [1]:
from sklearn.datasets import fetch_openml

# Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)

# The data is in the 'data' attribute and the target is in the 'target' attribute
x, y = mnist.data.to_numpy(), mnist.target.to_numpy()

print("Shape of data:", x.shape)
print("Shape of target:", y.shape)

Shape of data: (70000, 784)
Shape of target: (70000,)


In [2]:
from sklearn.model_selection import train_test_split,cross_val_score
x_train, x_test , y_train, y_test = train_test_split(x,y, train_size=0.85)

In [3]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()


In [4]:
import optuna

c:\Users\nisha\OneDrive - Indian Institute of Technology Guwahati\Documents\projects\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

import mlflow
import os

mlflow.set_experiment("MNIST-classifier")
mlflow.set_tracking_uri("file:///C:/Users/nisha/Documents/mlruns")  


2025/07/29 21:52:34 INFO mlflow.tracking.fluent: Experiment with name 'MNIST-classifier' does not exist. Creating a new experiment.


In [6]:
from sklearn.metrics import f1_score

In [7]:
import mlflow.sklearn
import sklearn

In [8]:
def objective(trial):
  n_estimator = trial.suggest_int('n_estimator',1,200)
  max_depth = trial.suggest_int('max_depth',2,10)
  min_samp_split = trial.suggest_int('min_samp_split',2,10)
  min_samples_leaf = trial.suggest_int('min_samples_leaf',2,10)

  clf = RandomForestClassifier(
      n_estimators= n_estimator,
      max_depth= max_depth,
      min_samples_split= min_samp_split,
      min_samples_leaf= min_samples_leaf,
      random_state = 42
  )


  clf.fit(x_train,y_train)
  y_pred = clf.predict(x_test)
  score = f1_score(y_test,y_pred, average= 'macro')
  
  with mlflow.start_run(nested=True):
        mlflow.log_params({
            "n_estimators": n_estimator,
            "max_depth": max_depth,
            "min_samples_split": min_samp_split,
            "min_samples_leaf":min_samples_leaf
        })
        mlflow.log_metric("f1_macro", float(score))
        mlflow.sklearn.log_model(clf, "model")
        
  return float(score)

In [10]:
from optuna.integration.mlflow import MLflowCallback

mlf_callback = MLflowCallback(
    tracking_uri=mlflow.get_tracking_uri(),
    metric_name="test_loss"  # match what you log
)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, callbacks=[mlf_callback])


C:\Users\nisha\AppData\Local\Temp\ipykernel_23816\2351544707.py:3: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlf_callback = MLflowCallback(
[I 2025-07-29 21:57:12,571] A new study created in memory with name: no-name-7ebf63ac-0d74-4a85-963e-8d9ad211e2a4
[W 2025-07-29 21:58:34,342] Trial 0 failed with parameters: {'n_estimator': 162, 'max_depth': 10, 'min_samp_split': 9, 'min_samples_leaf': 9} because of the following error: MlflowException('Could not find experiment with ID 704402935543956977').
Traceback (most recent call last):
  File "c:\Users\nisha\OneDrive - Indian Institute of Technology Guwahati\Documents\projects\venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\nisha\AppData\Local\Temp\ipykernel_23816\1580541608.py", line 20, in objective
    with mlflow.start_run(nested=True):
         ^^^^^^

MlflowException: Could not find experiment with ID 704402935543956977

In [ ]:
print(cross_val_score(clf,x_train,y_train,cv=3 , scoring='f1_macro'))

[0.96438972 0.96603527 0.96527719]
